In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets

In [2]:
def lagger(t, n, beta=2, sigma=4):
    l_0 = np.sqrt(sigma) * (np.exp(-beta * t / 2))
    l_1 = np.sqrt(sigma) * (1 - sigma * t) * (np.exp(-beta * t / 2))

    if n == 0:
        return l_0
    if n == 1:
        return l_1
    if n >= 2:
        l_next = (2 * 2 - 1 - t * sigma) / 2 * l_1 - (2 - 1) / 2 * l_0
        for j in range(3, n + 1):
            l_0 = l_1
            l_1 = l_next
            l_next = (2 * j - 1 - t * sigma) / j * l_1 - (j - 1) / j * l_0
        return l_next
t = widgets.BoundedIntText(
    value=7,
    min=0,
    max=1000,
    step=1,
    description='t:',
    disabled=False
)
n =widgets.BoundedIntText(
    value=5,
    min=0,
    max=1000,
    step=1,
    description='n:',
    disabled=False
)
widgets.interact(lagger, t=t, n=n, sigma=widgets.fixed(2), beta=widgets.fixed(4))

interactive(children=(BoundedIntText(value=7, description='t:', max=1000), BoundedIntText(value=5, description…

<function __main__.lagger(t, n, beta=2, sigma=4)>

In [3]:
def tabulate_lagger(T, n, beta, gamma):
    t = np.linspace(0, T, 10)
    results = lagger(t, n, beta, gamma)
    df = pd.DataFrame({'t': t, 'l': results})
    return df

n =widgets.BoundedIntText(
    value=5,
    min=0,
    max=1000,
    step=1,
    description='n:',
    disabled=False
)
beta =widgets.BoundedIntText(
    value=4,
    min=0,
    max=1000,
    step=1,
    description='beta:',
    disabled=False
)
gamma = widgets.BoundedIntText(
    value=2,
    min=0,
    max=1000,
    step=1,
    description="gamma:",
    disabled=False
)
widgets.interact(tabulate_lagger, T=widgets.fixed(10), n=n, beta=beta, gamma=gamma)

interactive(children=(BoundedIntText(value=5, description='n:', max=1000), BoundedIntText(value=4, description…

<function __main__.tabulate_lagger(T, n, beta, gamma)>

In [64]:
def experiment(n=20, beta=2, sigma=4, eps=0.001):
    t = 0
    while True:
        t += 0.0001
        res = []

        for i in range(n + 1):
            x = abs(lagger(t, n, beta, sigma))
            if x < eps:
                res.append(x)
                if i == n:
                    return t, pd.DataFrame({"results":res})
            else:
                break
n = widgets.BoundedIntText(
    value=20,
    min=0,
    max=100,
    step=1,
    description="n:",
    disabled=False
)
beta =widgets.BoundedIntText(
    value=4,
    min=0,
    max=1000,
    step=1,
    description='beta:',
    disabled=False
)
sigma = widgets.BoundedIntText(
    value=2,
    min=0,
    max=1000,
    step=1,
    description="sigma:",
    disabled=False
)
eps = widgets.FloatSlider(
    value=0.0001,
    min=0.00001,
    max=0.01,
    step=0.001,
    description="e:",
    disabled=False
)

In [65]:
a = widgets.interact(experiment, n=n, beta=beta, sigma=sigma, eps=eps)

interactive(children=(BoundedIntText(value=20, description='n:'), BoundedIntText(value=4, description='beta:',…

In [41]:
def quadratic(f, a, b, N=1000):
    
    x = np.linspace(a, b, N)
    s = sum([f(i) for i in x])
    return s*abs(b-a)/N

In [42]:
def lagger_transformation(f, n, beta=2, gamma=4):
    def integrand(t):
        return f(t)*lagger(t, n, beta, gamma)*np.exp(-t*(gamma-beta))
    b = experiment(100, beta, gamma)[0]

    return quadratic(integrand, 0, b)

In [43]:
func = lambda t: np.exp(-t)
print(quadratic(func, 0, 100,1000))
print(quadratic(func, 0, 100,2000))
print(quadratic(func, 0, 100,5000))

1.0498340282276697
1.0247084288590376
1.0098333397789774


In [44]:
def f(t):
    if 0 <= t <= 2 * np.pi:
        return np.sin(t - np.pi / 2) + 1
    elif t > 2 * np.pi:
        return 0

In [67]:
def tabulate_transformation(fi, n, t, eps):
    return pd.DataFrame({"results": [quadratic(f, t, i, eps) for i in range(n + 1)]})


n = widgets.BoundedIntText(
    value=20,
    min=0,
    max=100,
    step=1,
    description="n:",
    disabled=False
)
t = widgets.BoundedIntText(
    value=3,
    min=0,
    max=100,
    step=1,
    description="t:",
    disabled=False
)
eps = widgets.IntSlider(
    value=5,
    min=1,
    max=10,
    step=1,
    description="e:",
    disabled=False
)

widgets.interact(tabulate_transformation, fi=widgets.fixed(f), n=n, t=t, eps=eps)


interactive(children=(BoundedIntText(value=20, description='n:'), BoundedIntText(value=3, description='t:'), I…

<function __main__.tabulate_transformation(fi, n, t, eps)>

In [87]:
def reverse_lagger_transformation(lst, t, beta=2, sigma=4):
    return sum([lst["results"].values[i] * lagger(t, i) for i in range(len(lst["results"].values))])

transf_temp = tabulate_transformation(f, 30, 2, 4)
t = widgets.BoundedIntText(
    values=3,
    min=0,
    max=10,
    step=1,
    description="t:",
    disabled=False
)

widgets.interact(reverse_lagger_transformation, lst=widgets.fixed(transf_temp), t=t, beta=widgets.fixed(2), sigma=widgets.fixed(4))

interactive(children=(BoundedIntText(value=0, description='t:', max=10), Output()), _dom_classes=('widget-inte…

<function __main__.reverse_lagger_transformation(lst, t, beta=2, sigma=4)>

In [89]:
import matplotlib.pyplot as plt
def plot_lagger(T, N, beta=2, gamma=4):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    for n in range(N+1):
        lagger_value = tabulate_lagger(T, n, beta, gamma)
        ax.plot(lagger_value['t'], lagger_value['l'], label=f"n={n}", linewidth=1.0, alpha=0.8)
    
    ax.set_xlabel('T')
    ax.set_ylabel('Lagger Value')
    ax.legend()
    ax.grid(True)
    plt.show()

In [91]:
N = widgets.BoundedIntText(
    value=12,
    min=0,
    max=100,
    step=1,
    description="N:",
    disabled=False
)
T = widgets.BoundedIntText(
    value=3,
    min=0,
    max=100,
    step=1,
    description="T:",
    disabled=False
)

widgets.interact(plot_lagger, N=N, T=T)

interactive(children=(BoundedIntText(value=3, description='t:'), BoundedIntText(value=12, description='n:'), I…

<function __main__.plot_lagger(T, N, beta=2, gamma=4)>

In [96]:
def plot_transformation(f, n, beta=2, gamma=4):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()
    transform_values = tabulate_transformation(f, n, beta, gamma)
    ax.bar(range(0, n + 1), transform_values["results"].values, alpha=0.8, edgecolor='black')

    ax.set_xlabel("n")
    ax.set_ylabel("f")
    ax.set_title("Transformation Lagger")
    ax.set_xticks(range(1, n + 1))

    ax.grid(True, linestyle='--', alpha=0.7)

    plt.axhline(0, color='black', linewidth=1.0, linestyle='--')

    fig.tight_layout()
    plt.show()

In [99]:
n = widgets.BoundedIntText(
    value=12,
    min=0,
    max=100,
    step=1,
    description="n:",
    disabled=False
)

widgets.interact(plot_transformation, f=widgets.fixed(f), n=n)

interactive(children=(BoundedIntText(value=12, description='n:'), IntSlider(value=2, description='beta', max=6…

<function __main__.plot_transformation(f, n, beta=2, gamma=4)>

In [103]:
def plot_tranformations(f, n, beta=2, gamma=4, t1=0, t2=2*np.pi):

    transform_values = tabulate_transformation(f, n, beta, gamma)
    reversed_transform_values = [reverse_lagger_transformation(transform_values, t, beta, gamma) for t in np.linspace(t1, t2, 1000)]

    fig = plt.figure(figsize=(10, 10))
    ax = fig.gca()

    ax.plot(np.linspace(t1, t2, 1000), reversed_transform_values, alpha=0.5, linewidth=3.0)
    ax.set_xlabel("t")
    ax.set_ylabel("f(t)")
    ax.grid(True)
    ax.set_title("Reversed transformation")

    plt.show()

In [108]:
n = widgets.BoundedIntText(
    value=12,
    min=0,
    max=100,
    step=1,
    description="n:",
    disabled=False
)
t1 =  widgets.FloatSlider(
    value=0,
    min=-5*np.pi,
    max=5*np.pi,
    step=np.pi,
    description="t1:",
    disabled=False
)

widgets.interact(plot_tranformations, f=widgets.fixed(f), n=n, t1=t1)

interactive(children=(BoundedIntText(value=12, description='n:'), IntSlider(value=2, description='beta', max=6…

<function __main__.plot_tranformations(f, n, beta=2, gamma=4, t1=0, t2=6.283185307179586)>